# 2A.ml - Classification binaire avec features textuelles

Ce notebook propose de voir comment incorporer des features pour voir l'amélioration des performances sur une classification binaire. 

In [ ]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Récupérer les données

Les données sont téléchargeables [Compétition 2017 - additifs alimentaires](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/questions/competition_2A.html#id1) ou encore avec le code :

In [ ]:
from pyensae.datasource import download_data
data_train = download_data("off_train_all.zip", 
                           url="http://www.xavierdupre.fr/enseignement/complements/")

In [ ]:
data_test = download_data("off_test_all.zip", 
                           url="http://www.xavierdupre.fr/enseignement/complements/")

In [ ]:
import pandas
df = pandas.read_csv("off_test_all.txt", sep="\t", encoding="utf8", low_memory=False)

In [ ]:
df.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,hasE,s100
0,1.008255e+10,http://world-fr.openfoodfacts.org/produit/0010...,usda-ndb-import,1489064583,2017-03-09T13:03:03Z,1489064583,2017-03-09T13:03:03Z,"Golden Island, Pork Jerky, Grilled Barbecue",NaN,NaN,...,NaN,NaN,NaN,NaN,23.0,23.0,NaN,NaN,False,17.0
1,1.182204e+10,http://world-fr.openfoodfacts.org/produit/0011...,usda-ndb-import,1489070197,2017-03-09T14:36:37Z,1489070197,2017-03-09T14:36:37Z,"Big Fizz, Soda, Orange",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,7.0
2,2.548401e+10,http://world-fr.openfoodfacts.org/produit/0025...,usda-ndb-import,1489052024,2017-03-09T09:33:44Z,1489052024,2017-03-09T09:33:44Z,"Tofubaked Marinated Baked Tofu, Sesame Ginger",NaN,NaN,...,NaN,NaN,NaN,NaN,-2.0,-2.0,NaN,NaN,True,17.0
3,1.229250e+10,http://world-fr.openfoodfacts.org/produit/0012...,usda-ndb-import,1489133493,2017-03-10T08:11:33Z,1489133493,2017-03-10T08:11:33Z,Milk Chocolate Eggs,NaN,NaN,...,NaN,NaN,NaN,NaN,23.0,23.0,NaN,NaN,True,17.0
4,1.115054e+10,http://world-fr.openfoodfacts.org/produit/0011...,usda-ndb-import,1489052892,2017-03-09T09:48:12Z,1489052892,2017-03-09T09:48:12Z,Fresh Polish Sausage,NaN,NaN,...,NaN,NaN,NaN,NaN,22.0,22.0,NaN,NaN,True,17.0


In [ ]:
df.head(n=2).T[:50]

,0,1
code,1.00826e+10,1.1822e+10
url,http://world-fr.openfoodfacts.org/produit/0010...,http://world-fr.openfoodfacts.org/produit/0011...
creator,usda-ndb-import,usda-ndb-import
created_t,1489064583,1489070197
created_datetime,2017-03-09T13:03:03Z,2017-03-09T14:36:37Z
last_modified_t,1489064583,1489070197
last_modified_datetime,2017-03-09T13:03:03Z,2017-03-09T14:36:37Z
product_name,"Golden Island, Pork Jerky, Grilled Barbecue","Big Fizz, Soda, Orange"
generic_name,NaN,NaN
quantity,NaN,NaN


In [ ]:
df.head(n=2).T[50:100]

,0,1
ingredients_that_may_be_from_palm_oil,NaN,NaN
ingredients_that_may_be_from_palm_oil_tags,NaN,NaN
nutrition_grade_uk,NaN,NaN
nutrition_grade_fr,e,NaN
pnns_groups_1,NaN,NaN
pnns_groups_2,NaN,NaN
states,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed, en:nutrition-facts-complet..."
states_tags,"en:to-be-completed,en:nutrition-facts-complete...","en:to-be-completed,en:nutrition-facts-complete..."
states_fr,"A compléter,Informations nutritionnelles compl...","A compléter,Informations nutritionnelles compl..."
main_category,NaN,NaN


In [ ]:
df.head(n=2).T[100:150]

,0,1
cholesterol_100g,0.089,NaN
carbohydrates_100g,39.29,13.75
sugars_100g,39.29,13.75
sucrose_100g,NaN,NaN
glucose_100g,NaN,NaN
fructose_100g,NaN,NaN
lactose_100g,NaN,NaN
maltose_100g,NaN,NaN
maltodextrins_100g,NaN,NaN
starch_100g,NaN,NaN


In [ ]:
df.head(n=2).T[150:]

,0,1
caffeine_100g,NaN,NaN
taurine_100g,NaN,NaN
ph_100g,NaN,NaN
fruits-vegetables-nuts_100g,NaN,NaN
fruits-vegetables-nuts-estimate_100g,NaN,NaN
collagen-meat-protein-ratio_100g,NaN,NaN
cocoa_100g,NaN,NaN
chlorophyl_100g,NaN,NaN
carbon-footprint_100g,NaN,NaN
nutrition-score-fr_100g,23,NaN


## Exercice 1 : extraire un échantillon aléatoire

Les données sont volumineuses. Prenons un échantillon pour aller plus vite.

## Exercice 2 : caler un premier modèle de classification binaire

Avec les variables numériques uniquement. La cible est la variable ``hasE``.

## Exercice 3 : tracer une courbe ROC

## Exercice 4 : utiliser la variable qui contient le pays

Est-ce que le modèle est plus performant ?

## Exercice 5 : utiliser la variable additives_tags ?

Les résultats ne vous paraissent-ils pas étranges ?

## Exercice 6 : utiliser la variable ingredients_text ?

Que suggérez vous *embedding* ?

## Exercice 7 : utiliser la variable product_name ?

## Exercice 8 : utiliser GridSearch pour optimiser un hyper-paramètre du modèle ?

## Exercice 9 : faites tourner une cross validation ?